# Driving a transmon qubit 
In this tutorial we show how to drive a transmon qubit using pysqkit and how to extract the fidelity of the desired single-qubit gate. 

In [1]:
import numpy as np 
import time 
import matplotlib.pyplot as plt
from pysqkit.qubits import SimpleTransmon
from pysqkit.drives import microwave_drive
from pysqkit.drives.pulse_shapes import gaussian_top
from pysqkit.util.metrics import average_process_fidelity, average_gate_fidelity
from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.hsbasis import weyl_by_index
from pysqkit.solvers import solvkit
import qutip
import matplotlib 
matplotlib.rcParams['mathtext.fontset'] = 'cm'

A microwave gate for a transmon qubit is based on the following driven Hamiltonian 

$$H = H^{(0)}_T  + H_{drive},$$

$$H^{(0)}_T =  \hbar \omega_T \hat{b}^{\dagger}\hat{b} + \hbar \frac{\delta_T}{2} \hat{b}^{\dagger} \hat{b}^{\dagger}\hat{b} \hat{b},$$


$$\frac{H_{drive}}{h} = \varepsilon_T \cos(2\pi f_d t) \hat{n}_T.$$

The (approximate) charge operator for the transmon in terms of annihilation and creation operators reads

$$
\hat{n}_T = i \biggl( \frac{E_J}{32 |\delta_T |} \biggr)^{1/4} (b^{\dagger} - b),
$$
where 
$$
E_J = \hbar \biggl(\frac{\omega_T - \delta_T}{8 | \delta_T |} \biggr)^2.
$$
Let us introduce the transmon object

In [2]:
temperature = 0.020 #K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz

levels_t = 3
transm = SimpleTransmon(
    label='T',
    max_freq=5.0,
    anharm=-0.3,
    diel_loss_tan=0.75*1e-6,
    env_thermal_energy=thermal_energy,
    dim_hilbert=levels_t
)

#We also add the drive 
transm.add_drive(microwave_drive, label='sq_gate', pulse_shape=gaussian_top)

In [3]:
# Setting drive parameters
eps_drive = 0.03
freq_drive = transm.freq
t_tot = 40 #ns 
t_rise = 15 #ns
pts_per_drive_period = 5 
nb_points = int(t_tot*freq_drive*pts_per_drive_period)
t_list = np.linspace(0, t_tot, nb_points)
transm.drives['sq_gate'].set_params(phase=0.0, time=t_list, rise_time=t_rise, amp=eps_drive, freq=freq_drive)

In [4]:
# Setting up simulation
hamil0 = transm.hamiltonian(as_qobj=True)
hamil_drive = [transm.drives['sq_gate'].hamiltonian(as_qobj=True)]
pulse_drive = [transm.drives['sq_gate'].eval_pulse()]
jump_ops = transm.collapse_ops(as_qobj=True)
state_in = transm.state('0', as_qobj=True)[1]

output = solvkit.integrate(2*np.pi*t_list, state_in, hamil0, hamil_drive, pulse_drive, jump_ops, "mesolve") 


